## Histopathologic CNN Analyzer

This architecture is based on the AlexNet network as described in Dive Into Deep Learning (2021, Aston Zhang, Zachary C. Lipton, Mu Li, and Alexander J. Smola), Section 6.6.1.

It has been slightly modified to account for the smaller image sizes.

In [ ]:
import torch.nn as nn
import torchvision.transforms as tx

In [ ]:
class Histopath_CNN(nn.Module):

    def __init__(self, debug):
        super(Histopath_CNN, self).__init__()
        
        self.debug = debug

        # Image Pre-Processing
        self.conv0 = nn.Sequential(
            nn_debug(debug),
            # Add standard center cropping based on numerous trials of larger and 
            #   smaller data values for the Histopathologic Cancer dataset.
            #   The labelled portion of the images are within the 32x32 center.
            tx.CenterCrop(54)
        )

        self.conv = nn.Sequential(
            # Print input shape
            nn_debug(debug),
            # Conv Layer 1
            nn.Conv2d(3, 96, kernel_size=7, stride=1, padding=2),
            nn_debug(debug),
            # Activation function - non-linear function - ReLU offers better 
            #  training performance due to its less gentle curve than other activations
            nn.BatchNorm2d(96),
            nn.Tanh(),
            # Pooling 'steps back' from the details a little bit in order to help
            #   generalize the image
            nn.MaxPool2d(kernel_size=4, stride=1, padding=2),
            nn_debug(debug),

            # Conv Layer 2
            nn.Conv2d(96, 192, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(192),
            nn_debug(debug),
            nn.Tanh(),
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),

            # Three consecutive convo layers
            #nn_debug(debug),
            nn.Conv2d(192, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.Tanh(),

            #nn_debug(debug),
            #nn.Conv2d(128, 32, kernel_size=3, stride=1, padding=1),
            #nn.BatchNorm2d(32),
            #nn.Tanh(),

            nn_debug(debug),
            nn.Conv2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.Tanh(),

            nn_debug(debug),
            nn.MaxPool2d(kernel_size=4, stride=2, padding=1),
            # Collapse the 3D data into a 1D input for the linear units
            nn_debug(debug),
            nn.Flatten()
        )

        # Traditional linear network for classification learning
        self.fc = nn.Sequential(
            nn_debug(debug),

            # FC Layer 1
            nn.Linear(2304, 2880),
            nn.ReLU(),
            #nn.Dropout(p=0.5),

            # FC Layer 2
            nn.Linear(2880, 1728),
            nn.ReLU(),
            #nn.Dropout(p=0.25),

            # Output Layer
            nn.Linear(1728, 1),
            # Apply Sigmoid activation in order to get a smoother probability
            #   curve, which is required by the BinaryCrossEntropy Loss function
            nn.Sigmoid()
        )

    def forward(self, x):
        # Image tx
        x = self.conv0(x)

        # convolutions
        x = self.conv(x)

        # FC layers
        x = self.fc(x)

        return x

In [ ]:
class nn_debug(nn.Module):
    def __init__(self, debug):
        super(nn_debug, self).__init__()
        self.debug = debug

    def forward(self, x):
        if self.debug:
            print(f'## Input: {x.shape}')
        return x